In [2]:
import os
import pathlib
import shutil
import zipfile

In [3]:
from tkinter import Tk
from tkinter.filedialog import askdirectory

# Hide the main Tkinter window
Tk().withdraw()
# Print the selected folder
print(f"Please select the folder from the pop up:")
# Ask the user to select the folder with custom text
home_folder = askdirectory(title="Select the Project_submission folder")

# Check if a folder was selected
if not home_folder:
    print("No folder selected. Exiting.")
    exit()

# Print the selected folder or use it in your script
print(f"Selected folder: {home_folder}")


Please select the folder from the pop up:
Selected folder: D:/Others/Project_Submission_ALL_DATA


In [4]:
import os
first_folder = os.path.join(home_folder,'DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS')

def walk_through_dir(dir_path):
  count_images = 0
  for dirpath, dirnames, filenames in os.walk(dir_path):
    count_images += len(filenames)
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
  print(f"Total Images : {count_images}")
walk_through_dir(first_folder)

There are 3 directories and 0 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS'.
There are 15 directories and 0 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS\test'.
There are 0 directories and 28 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS\test\up to 10 year'.
There are 0 directories and 27 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS\test\up to 11 year'.
There are 0 directories and 27 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS\test\up to 12 year'.
There are 0 directories and 25 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS\test\up to 13 year'.
There are 0 directories and 25 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle

In [5]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from torchvision.models import VisionTransformer
from tqdm import tqdm
import time
import os
import shutil
import timm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import numpy as np
import seaborn as sns

c:\Users\Chinmay\anaconda3\envs\tf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def generate_loader( train_dir , test_dir , val_dir):

    batch_size = 32
    train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    num_classes = len(train_dataset.classes)

    return train_loader , test_loader , val_loader

In [7]:

def load_pretrained_vision_transformer(model_name, num_classes, pretrained=True):
    # Load pre-trained Vision Transformer model
    model = timm.create_model(model_name, pretrained=pretrained)

    # Modify classifier head for the specified number of classes
    if hasattr(model, 'head'):
        num_features = model.head.in_features
        model.head = torch.nn.Linear(num_features, num_classes)
    elif hasattr(model, 'classifier'):
        num_features = model.classifier.in_features
        model.classifier = torch.nn.Linear(num_features, num_classes)
    else:
        raise ValueError("Unsupported model architecture, unable to modify classifier head.")

    return model




In [8]:


def test_model(model, test_loader, batch_size=32):


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = correct / total
    print(f'Testing Accuracy: {test_accuracy:.4f}')
    return test_accuracy




def save_accuracy_curves( train_accuracies, val_accuracies, save_path):
    plt.figure(figsize=(4 , 4))
    plt.rcParams['pdf.fonttype']=42
    plt.rcParams['savefig.format'] = 'eps'
    plt.rcParams['savefig.dpi'] = 420
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = 'Arial'
    plt.plot(train_accuracies, label='Training Accuracy', color='blue')
    plt.plot(val_accuracies, label='Validation Accuracy', color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Vision Transformer')
    plt.legend()
    plt.grid(True)
    # Save the image of accuracy curves
    #plt.savefig(save_path , format= 'png' , dpi = 420)
    plt.show()
    





In [9]:

def calculate_metrics(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')

***Vision Transformer Classfication Report without segmentation for 15 Classes***

In [10]:
# Example usage:
model_name = 'vit_base_patch16_224.augreg_in21k'
num_classes = 15  # Assuming the number of classes is known
pretrained = True   # Set to False if you don't want to use pre-trained weights
model = load_pretrained_vision_transformer(model_name, num_classes, pretrained)

# Load the saved state dictionary
state_dict = torch.load(os.path.join(home_folder,'RESULTS/MODEL_FILES/Model_Files_15_Classes/vision_transformer_model_15_classes_ws.pth') , map_location='cpu')

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Ensure the model is in evaluation mode
model.eval()


# Now your model is loaded and ready to be used for inference

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False

In [11]:
train_dir = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS/train")
val_dir = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS/val")
test_dir = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 15 Classes WS/test")

In [12]:
train_loader , test_loader , val_loader = generate_loader( train_dir , test_dir , val_dir)

In [13]:
test_accuracy = test_model(model, test_loader, batch_size=32)
print(f"Accuracy : {test_accuracy}")

Testing Accuracy: 0.3408
Accuracy : 0.34075723830734966


In [14]:
calculate_metrics(model, test_loader)

Precision: 0.3356, Recall: 0.3408, F1-score: 0.3348


***Vision Transformer Classfication Report without segmentation for 4 Classes***

In [17]:
first_folder = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS")
walk_through_dir(first_folder)



There are 3 directories and 0 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS'.
There are 4 directories and 0 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS\test'.
There are 0 directories and 144 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS\test\Group 1 (1 to 5 years)'.
There are 0 directories and 135 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS\test\Group 2 (5.1 to 9 years)'.
There are 0 directories and 103 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS\test\Group 3 (9.1 to 13 years)'.
There are 0 directories and 53 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS\test\Group 4 (13.1 and above years)'.
There are 4 directories and 0 images in 'D:/Others/Project_Submis

In [20]:
# Example usage:
model_name = 'vit_base_patch16_224.augreg_in21k'
num_classes = 4  # Assuming the number of classes is known
pretrained = True   # Set to False if you don't want to use pre-trained weights
model = load_pretrained_vision_transformer(model_name, num_classes, pretrained)

# Load the saved state dictionary
state_dict = torch.load(os.path.join(home_folder,'RESULTS/MODEL_FILES/Model_Files_4_Classes/vision_transformer_model_4_classes_ws.pth') , map_location='cpu')

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Ensure the model is in evaluation mode
model.eval()


# Now your model is loaded and ready to be used for inference

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False

In [21]:
train_dir = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS/train")
val_dir = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS/val")
test_dir = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 4 Classes WS/test")

In [22]:
train_loader , test_loader , val_loader = generate_loader( train_dir , test_dir , val_dir)

In [23]:
test_accuracy = test_model(model, test_loader, batch_size=32)
print(f"Accuracy : {test_accuracy}")

Testing Accuracy: 0.6943
Accuracy : 0.6942528735632184


In [24]:
calculate_metrics(model, test_loader)

Precision: 0.6934, Recall: 0.6943, F1-score: 0.6934


***Vision Transformer Classfication Report without segmentation for 7 Classes***

In [25]:
first_folder = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS")
walk_through_dir(first_folder)


There are 3 directories and 0 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS'.
There are 7 directories and 0 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS\test'.
There are 0 directories and 67 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS\test\Group 1 (1 to 3 years)'.
There are 0 directories and 79 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS\test\Group 2 (3.1 to 5 years)'.
There are 0 directories and 72 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS\test\Group 3 (5.1 to 7 years)'.
There are 0 directories and 64 images in 'D:/Others/Project_Submission_ALL_DATA\DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS\test\Group 4 (7.1 to 9 years)'.
There are 0 directories and 54 images in 'D:/Others/Project_Submission_ALL_

In [26]:
# Example usage:
model_name = 'vit_base_patch16_224.augreg_in21k'
num_classes = 7  # Assuming the number of classes is known
pretrained = True   # Set to False if you don't want to use pre-trained weights
model = load_pretrained_vision_transformer(model_name, num_classes, pretrained)

# Load the saved state dictionary
state_dict = torch.load(os.path.join(home_folder,'RESULTS/MODEL_FILES/Model_Files_7_Classes/vision_transformer_model_7_classes_ws.pth') , map_location='cpu')

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Ensure the model is in evaluation mode
model.eval()


# Now your model is loaded and ready to be used for inference

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False

In [27]:
train_dir = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS/train")
val_dir = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS/val")
test_dir = os.path.join(home_folder,"DATASET_WS_VERIFICATION/Cattle Dataset 7 Classes WS/test")

In [28]:
train_loader , test_loader , val_loader = generate_loader( train_dir , test_dir , val_dir)

In [29]:
test_accuracy = test_model(model, test_loader, batch_size=32)
print(f"Accuracy : {test_accuracy}")

Testing Accuracy: 0.5604
Accuracy : 0.5603644646924829


In [ ]:
calculate_metrics(model, test_loader)

Precision: 0.5631, Recall: 0.5604, F1-score: 0.5609
